# Locally testing the append logic

<div style="border-left: 5px solid #f39c12; background-color: #fcf8e3; padding: 0.75em; margin: 1em 0;">
  <strong style="color:#c09853;">⚠️ WARNING:</strong>  
  If you run this on the staging/production store, it will create at least snapshots/branches on the store and possibly changes to the main branch (depending on the <code>DRY_RUN</code> environment variable). So proceed with caution.
</div>

To run this example in an existing Jupyter environment (e.g. a Jupyter Hub) please make sure to create a custom kernel with uv fowllowing these [instructions](../README.md).

In [1]:
from lambda_function import write_to_icechunk_or_fail, open_xr_dataset_from_branch, open_icechunk_repo

/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# This is a temporary store in a scratch bucket, which will be deleted after 7 days.
# You can recreate the test store by running `./scripts/rebuild_store_with_vz2_ic1`
from dotenv import load_dotenv
import os
load_dotenv()
# store_url = os.environ["ICECHUNK_STORE_DIRECT"]
store_url = os.environ["ICECHUNK_STORE_PROD"]

repo = open_icechunk_repo(store_url)

opening icechunk repo
Defining icechunk storage for s3


In [3]:
# Look at the store before
ds_before = open_xr_dataset_from_branch(repo, 'main')
ds_before

/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


<xarray.Dataset> Size: 8TB
Dimensions:           (lon: 36000, time: 436, lat: 17999)
Coordinates:
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * time              (time) datetime64[ns] 3kB 2024-06-02T09:00:00 ... 2025-...
Data variables:
    analysis_error    (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysed_sst      (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 1TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    ...                         ...
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

In [5]:
# update the store
os.environ['DRY_RUN']='true'
write_to_icechunk_or_fail(store_url, parallel=False)

opening icechunk repo
Defining icechunk storage for s3
Finding dates to append to existing store


/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


limit_granules=None
Searching for granules between 2025-08-11 21:00:01 and 2025-08-15 21:00:00
Number of granules found: 3
podaac-ops-cumulus-protected


/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)
/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/virtualizarr/xarray.py:341: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  with xr.open_zarr(
/home/jovyan/mursst-icechunk-up

New Data (Virtual): <xarray.Dataset> Size: 12GB
Dimensions:           (time: 3, lat: 17999, lon: 36000)
Coordinates:
  * time              (time) datetime64[ns] 24B 2025-08-12T09:00:00 ... 2025-...
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
Data variables:
    analysed_sst      (time, lat, lon) int16 4GB ManifestArray<shape=(3, 1799...
    analysis_error    (time, lat, lon) int16 4GB ManifestArray<shape=(3, 1799...
    mask              (time, lat, lon) int8 2GB ManifestArray<shape=(3, 17999...
    sea_ice_fraction  (time, lat, lon) int8 2GB ManifestArray<shape=(3, 17999...
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Interim near-real-time (nrt) p...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
   

/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


Commit successful to branch: add_time_2025-08-15T21:18:09.061395+00:00 as snapshot:ZAQZNVSADF9AS16W8K90 
 MUR update add_time_2025-08-15T21:18:09.061395+00:00
Reloading Dataset from branch
Dataset on add_time_2025-08-15T21:18:09.061395+00:00: <xarray.Dataset> Size: 8TB
Dimensions:           (lat: 17999, time: 439, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 4kB 2024-06-02T09:00:00 ... 2025-...
Data variables:
    analysed_sst      (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 1TB dask.array<chunksize=(1, 1023, 2047)

QUEUEING TASKS | : 100%|██████████| 3/3 [00:00<00:00, 309.88it/s]
PROCESSING TASKS | : 100%|██████████| 3/3 [00:00<00:00,  3.19it/s]
COLLECTING RESULTS | : 100%|██████████| 3/3 [00:00<00:00, 54947.21it/s]
/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packages/dask/_task_spec.py:758: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  return self.func(*new_argspec, **kwargs)
/home/jovyan/mursst-icechunk-updater/.venv/lib/python3.12/site-packag

Test Dataset: <xarray.Dataset> Size: 54GB
Dimensions:           (time: 3, lat: 17999, lon: 36000)
Coordinates:
  * time              (time) datetime64[ns] 24B 2025-08-12T09:00:00 ... 2025-...
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
Data variables:
    analysed_sst      (time, lat, lon) float64 16GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 16GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 8GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 16GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Interim near-real-time (nrt) p...
    summary:                    A merged, multi-sensor L4 Foundation

'Success'